# train_landmark_localiser

Train the landmark localiser for Zeno using dlib's implementation of the ensemble of regression trees (ERT) methods.

## Step 1: Import models

In [97]:
import os
import cv2
import glob
import dlib
import time
import pandas as pd
try:
    from ConfigParser import ConfigParser    # If using Python 2.7
except ImportError:
    from configparser import ConfigParser    # If using Python 3.5
config = ConfigParser()
config.read('config.ini')
print('All modules imported.')

All modules imported.


## Step 2: Prepare the data structure for training

In [98]:
# Load images and annotation
annotations = pd.read_pickle(os.path.realpath(os.path.join('./dataset', 'annotations.pkl')))
images = []
groundtruth = []
last_check_time = time.time()
for idx in range(annotations.shape[0]):
    entry = annotations.iloc[idx]
    image_path = os.path.realpath(os.path.join('./dataset', entry['session'], 
                                               '%06d.png' % entry['index']))
    images.append(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2GRAY))
    bounding_box = dlib.rectangle(entry['face_box'][0], entry['face_box'][1], 
                                  entry['face_box'][0] + entry['face_box'][2] - 1, 
                                  entry['face_box'][1] + entry['face_box'][3] - 1)
    landmarks = [dlib.point(int(round(pts[0])), int(round(pts[1]))) 
                 for pts in entry['landmarks']]
    groundtruth.append([dlib.full_object_detection(bounding_box, landmarks)])
    current_time = time.time()
    if last_check_time < current_time - 10.0:
        last_check_time = current_time
        print('%d samples have been prepared.' % (idx + 1))
print('All %d samples have been prepared.' % annotations.shape[0])

All 349 samples have been prepared.


## Step 3: Train the shape predictor model

In [99]:
options = dlib.shape_predictor_training_options()
options.be_verbose = config.getboolean('shape_predictor_training_options', 
                                       'be_verbose')
options.cascade_depth = config.getint('shape_predictor_training_options', 
                                      'cascade_depth')
options.feature_pool_region_padding = config.getfloat('shape_predictor_training_options', 
                                                      'feature_pool_region_padding')
options.feature_pool_size = config.getint('shape_predictor_training_options', 
                                          'feature_pool_size')
options.lambda_param = config.getfloat('shape_predictor_training_options', 
                                       'lambda_param')
options.nu = config.getfloat('shape_predictor_training_options', 'nu')
options.num_test_splits = config.getint('shape_predictor_training_options', 
                                        'num_test_splits')
options.num_trees_per_cascade_level = config.getint('shape_predictor_training_options', 
                                                    'num_trees_per_cascade_level')
options.oversampling_amount = config.getint('shape_predictor_training_options', 
                                            'oversampling_amount')
options.tree_depth = config.getint('shape_predictor_training_options', 'tree_depth')
shape_predictor = dlib.train_shape_predictor(images, groundtruth, options)
shape_predictor_model_path = os.path.realpath(os.path.join('./models', 'zeno_face_tracker_model.dat'))
shape_predictor.save(shape_predictor_model_path)
print('Shape predictor model has been trained and saved to: ' + shape_predictor_model_path)

Shape predictor model has been trained and saved to: D:\hhj\zeno_face_tracker\models\zeno_face_tracker_model.dat


## Step 4: Test the model on live video

In [92]:
model.save('lala.dat')n